<a href="https://colab.research.google.com/github/Annie00000/Project/blob/main/4_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Html.Form

在Dash中，html.Form对应于HTML中的 "form" 元素，它用于创建一个包含用户输入元素的**表单**，例如文本字段、复选框、单选按钮、提交按钮等。用户可以填写表单，然后提交表单数据。表单提交时，可以配置Dash应用以异步方式（使用回调）处理表单数据，而无需重新加载页面。

* 特性

1. 用户输入收集：html.Form主要用于收集用户输入的数据。
2. 支持多种输入类型：可以包含各种html组件作为表单元素，比如dcc.Input, html.Select, dcc.Checklist等。
3. 数据提交处理：表单提交可以通过按钮触发，然后通过Dash的回调机制处理提交的数据。

* 使用方法和注意事项

1. 创建表单：在Dash应用中，你可以使用html.Form组件来创建一个表单。你可以将输入组件（如dcc.Input）和一个提交按钮（html.Button）放在html.Form元素内部。

2. 表单提交：表单的数据可以通过用户点击提交按钮来提交。你可以通过定义一个回调函数来处理表单的提交事件，通常是监听按钮的n_clicks属性。

3. 防止页面刷新：在传统的HTML中，表单提交通常会导致页面刷新或跳转。在Dash中，通过异步回调处理表单提交可以避免这一点，为了防止页面刷新，不需要设置action属性。

4. 使用prevent_default：在回调中处理表单提交时，为了防止默认的表单提交行为（即页面刷新），可以在回调装饰器中使用prevent_default=True参数。这样，即使在用户提交表单时，也不会导致页面刷新。

5. 数据收集：在回调函数中，你可以通过dash.dependencies.State来访问表单内各输入组件的值，而不是通过Input。因为通常你希望在用户点击提交按钮时一次性获取所有表单数据，而不是在数据改变时即时获取。

## 2.

* id : 識別符，唯一
* type : 属性，指定输入字段的类型，比如文本（text）、密码（password）、邮件（email）等。此外，在其他一些组件中，type也可能用于定义组件的变体或行为，但这取决于具体的组件。

## 3.

我有page1和page2，而page2是由page1選取的參數製造而成的頁面，但我目前想讓兩個分頁同時存在，也就是說當我page2有資料了，在切回page1的時候，page2的東西不會因此消失，可以用原本page1選取的參數當作page2的URL，然後可以重複顯現出。

1. 使用dcc.Location组件来监听URL的变化，并作为页面内容的输入。
2. 当用户从page1提交参数时，更新URL的查询字符串，以包含这些参数。
3. 在page2的布局和回调中，根据URL的查询字符串来渲染内容。

In [ ]:
from dash import Dash, dcc, html, Input, Output, State, callback_context

app = Dash(__name__)
app.layout = html.Div([
    dcc.Location(id='url', refresh=False),
    html.Div(id='page-content')
])

# Page1 Layout
page1_layout = html.Div([
    dcc.Input(id='input-on-page1', type='text', placeholder='输入一些参数'),
    html.Button('提交到Page2', id='submit-button', n_clicks=0),
])

# Page2 Layout Function
def page2_layout(params):
    return html.Div([
        html.H1('Page2'),
        html.P(f'从Page1传递的参数: {params}')
    ])

@app.callback(Output('page-content', 'children'),
              [Input('url', 'pathname'), Input('url', 'search')],
              [State('input-on-page1', 'value')])
def display_page(pathname, search, input_value):
    ctx = callback_context
    if not ctx.triggered or ctx.triggered[0]['prop_id'] == 'url.pathname':
        if pathname == '/page1':
            return page1_layout
        elif pathname == '/page2':
            # 从URL查询字符串中解析参数
            params = search.lstrip('?')
            return page2_layout(params)
        else:
            return '404'
    elif ctx.triggered[0]['prop_id'].endswith('.n_clicks'):
        # 更新URL，将参数作为查询字符串
        params = f'?{input_value}'
        return dcc.Location(pathname='/page2', search=params)

if __name__ == '__main__':
    app.run_server(debug=True)


在这个例子中：

当用户在page1中输入参数并点击提交时，应用会导航到page2，并在URL中包含输入的参数作为查询字符串。

page2的布局函数page2_layout会接受这些参数，并根据这些参数渲染内容。
由于参数存储在URL中，用户在页面之间切换时，page2的状态（基于page1的选择）会被保留。

这种方法的好处是，它允许参数在URL中明确可见，便于分享和书签页面，同时保留页面状态，即使在用户刷新页面或在页面间切换时也不会丢失。